In [ ]:
# Plot the displacement field around the tunnel computed in the Moose framework
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import os
import sys
from pathlib import Path


# Path to repo root
repo_root = Path.cwd().parents[1] 

# Path to data file
data_file = repo_root / "synthetic_data" / "TransverseIsotropyShear_out_point_sample_0001.csv"

if not data_file.exists():
    raise FileNotFoundError(f"Fichier de données introuvable : {data_file}")


df = pd.read_csv(data_file)

R = 5   
L = 24   

x = df["x"]
y = df["y"]

# Domain mask
mask = (x <= (R + L)) & (x >= (-R - L)) & (y >= (-R - L)) & (y <= (R + L))

df_filtered = df[mask].copy()

R = 5
x = df_filtered["x"].to_numpy()
y = df_filtered["y"].to_numpy()
ux = df_filtered["disp_x"].to_numpy()
uy = df_filtered["disp_y"].to_numpy()

# Regular meshgrid
nx, ny = 500, 500  
xi = np.linspace(x.min(), x.max(), nx)
yi = np.linspace(y.min(), y.max(), ny)
X, Y = np.meshgrid(xi, yi)

# Interpolate scattered data onto the grid
UX = griddata((x, y), ux, (X, Y), method='linear')
UY = griddata((x, y), uy, (X, Y), method='linear')

# Mask points inside the circle
mask = X**2 + Y**2 <= R**2
UX[mask] = np.nan
UY[mask] = np.nan

def plot_grid_disp(X, Y, U, label, save_path):

    plt.rcParams.update({
        "font.size": 6,
        "axes.labelsize": 6,
        "xtick.labelsize": 6,
        "ytick.labelsize": 6,
        "legend.fontsize": 6
    })
    
    fig, ax = plt.subplots(figsize=(5/2.54*1.25, 5/2.54*1.25))
    cf = ax.contourf(X, Y, U, levels=50, cmap="RdBu_r", extend="both")

    # Colorbar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="6%", pad=0.1)
    cbar = plt.colorbar(cf, cax=cax)
    
    # Scientific notation
    formatter = ScalarFormatter(useMathText=True)
    formatter.set_powerlimits((0, 0))  
    cbar.ax.yaxis.set_major_formatter(formatter)
    
    offset_text = cbar.ax.yaxis.get_offset_text()
    offset_text.set_fontsize(6)
    offset_text.set_x(3.5)  
    
    cbar.ax.tick_params(labelsize=6)

   # Units
    plt.text(3.60, 1.09, "(m)", transform=plt.gca().transAxes)

    # Tunnel 
    circle = plt.Circle((0,0), R, color="red", fill=False, linewidth=1, linestyle="-")
    ax.add_patch(circle)

    ax.set_xlabel(r"$x$ (m)")
    ax.set_ylabel(r"$y$ (m)")
    ax.axis("equal")
    ax.grid(True)
    plt.tight_layout()
    plt.savefig(f"{save_path}/{label}_true.pdf", dpi=300, bbox_inches="tight")
    plt.show()

# Plot ux and uy
save_dir = os.path.join(repo_root, "moose")

plot_grid_disp(X, Y, UX, "ux", save_dir)
plot_grid_disp(X, Y, UY, "uy", save_dir)